## PFDA - Contact Center Performance Analysis Project

## Section 1 - EDA

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
df = pd.read_excel('pwc_call_centre_data.xlsx')
df.set_index('Date', inplace=True)
df.head()

,Call Id,Agent,Time,Topic,Answered (Y/N),Resolved,Speed of answer in seconds,AvgTalkDuration,Satisfaction rating
Date,,,,,,,,,
2021-01-01,ID0001,Diane,09:12:58,Contract related,Y,Y,109.0,00:02:23,3.0
2021-01-01,ID0002,Becky,09:12:58,Technical Support,Y,N,70.0,00:04:02,3.0
2021-01-01,ID0003,Stewart,09:47:31,Contract related,Y,Y,10.0,00:02:11,3.0
2021-01-01,ID0004,Greg,09:47:31,Contract related,Y,Y,53.0,00:00:37,2.0
2021-01-01,ID0005,Becky,10:00:29,Payment related,Y,Y,95.0,00:01:00,3.0


In [14]:
# Convert all column names to lowercase
df.columns = df.columns.str.lower()


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 2021-01-01 to 2021-03-31
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Call Id                     5000 non-null   object 
 1   Agent                       5000 non-null   object 
 2   Time                        5000 non-null   object 
 3   Topic                       5000 non-null   object 
 4   Answered (Y/N)              5000 non-null   object 
 5   Resolved                    5000 non-null   object 
 6   Speed of answer in seconds  4054 non-null   float64
 7   AvgTalkDuration             4054 non-null   object 
 8   Satisfaction rating         4054 non-null   float64
dtypes: float64(2), object(7)
memory usage: 390.6+ KB


In [10]:
df.describe()

,Speed of answer in seconds,Satisfaction rating,AvgTalkDuration_seconds
count,4054.000000,4054.000000,4054.000000
mean,67.520720,3.403552,224.922792
std,33.592872,1.212220,111.381555
min,10.000000,1.000000,30.000000
25%,39.000000,3.000000,130.000000
50%,68.000000,3.000000,226.000000
75%,97.000000,4.000000,319.000000
max,125.000000,5.000000,420.000000


In [30]:
# Count the occurrences of each unique topic
topic_counts = df['topic'].value_counts()
topic_counts

topic
Streaming            1022
Technical Support    1019
Payment related      1007
Contract related      976
Admin Support         976
Name: count, dtype: int64

## Section 2 - Data Cleaning

To analyse the data efficiently and see if there is a correlation between speed of answer in seconds and AvgTalkDuration they most both be in seconds. Pandas documentation shows how to convert minutes into seconds https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.total_seconds.html

Besides AverageTalkDuration the other data types are in order so no more adjustments to be made.

In [19]:
# Ensure 'AvgTalkDuration' is in string format
df['avgtalkduration'] = df['avgtalkduration'].astype(str)

# Convert 'AvgTalkDuration' to seconds
df['avgtalkduration_seconds'] = pd.to_timedelta(df['avgtalkduration']).dt.total_seconds()


There are three columns with 946 null values each. When running df.head(50) there is a clear trend where the null values are coming from, when the call isn't answered. Neither dropna() or fillna() would be benificial for analysis so null values are being kept as is.

In [16]:
# Check are there null values to deal with
df.isnull().sum()

call id                         0
agent                           0
time                            0
topic                           0
answered (y/n)                  0
resolved                        0
speed of answer in seconds    946
avgtalkduration                 0
satisfaction rating           946
avgtalkduration_seconds       946
dtype: int64

In [22]:
# Check for duplicates
duplicates = df.duplicated().sum()
duplicates

0

In [28]:
# Drop Irrelevant Columns
df = df.drop(columns=['call id'])
df.head()

,agent,time,topic,answered (y/n),resolved,speed of answer in seconds,avgtalkduration,satisfaction rating,avgtalkduration_seconds
Date,,,,,,,,,
2021-01-01,Diane,09:12:58,Contract related,Y,Y,109.0,00:02:23,3.0,143.0
2021-01-01,Becky,09:12:58,Technical Support,Y,N,70.0,00:04:02,3.0,242.0
2021-01-01,Stewart,09:47:31,Contract related,Y,Y,10.0,00:02:11,3.0,131.0
2021-01-01,Greg,09:47:31,Contract related,Y,Y,53.0,00:00:37,2.0,37.0
2021-01-01,Becky,10:00:29,Payment related,Y,Y,95.0,00:01:00,3.0,60.0
